In [48]:
import numpy as np
import pandas as pd
import os
import cv2
from mtcnn.mtcnn import MTCNN
detector = MTCNN()
def cropFaces(img):
    
    # detect faces in the image
    faces = detector.detect_faces(img)
    #print(faces)
    if len(faces) == 0:
        return faces
    x, y, width, height = faces[0]['box']
    #img[y:y+height , x:x+width, :]
    return img[y:y+height, x:x+width, :]

In [49]:
from keras.utils import to_categorical
users = {'ben_afflek':0,'elton_john':1,'jerry_seinfeld':2,'madonna':3,'mindy_kaling':4}
images = []
label = []
for dirname, _, filenames in os.walk('/Users/91818/Downloads/ML_Project/archive/data/train/'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cropFaces(img)
        if(img.shape[0] == 0 or img.shape[1] == 0 or len(img) == 0):
            continue
        img = cv2.resize(img,(224,224))/255
        images.append(img)
        lbl = dirname.split('/')[-1]
        label.append(users[lbl])

df = pd.DataFrame()   #we are creating this dataframe to shuffle the input dataset.
df['image'] = images
df['label'] = label
df = df.sample(frac = 1)
X_train=np.array(df.iloc[:,0].tolist())
y_train = np.array(df.iloc[:,1].tolist())
y_train = to_categorical(y_train)

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 19ms/step


In [105]:
# function for creating a block
bnEps = 2e-5
bnMom = 0.9
chanDim = 1

def resnet(layer_in, n_filters,s):
    data = layer_in
    stride = (s,s)
    
    #1st Convolutional Layer
    merge_input = Conv2D(n_filters, (1,1),strides = (s,s))(layer_in)        
    bn2 = BatchNormalization(axis = chanDim,epsilon=bnEps,momentum = bnMom)(merge_input)
    act2 = Activation('relu')(bn2)
    
    #2nd Convolutional Layer
    conv2 = Conv2D(n_filters, (3,3),strides= stride,use_bias = False, padding='same', kernel_initializer='he_normal')(act2)  
    bn3 = BatchNormalization(axis = chanDim,epsilon=bnEps,momentum = bnMom)(conv2)
    act3 = Activation('relu')(bn3)
    
    #3rd Convolutional layer
    conv3 = Conv2D(n_filters, (1,1),use_bias = False, kernel_initializer='he_normal')(act3) 
    
    data = Conv2D(n_filters,(1,1),padding = 'valid',strides = (s,s))(data) # Adjusting the input size according to 3rd convolutional layer
    data = BatchNormalization()(data)  
    # add filters, assumes filters/channels last
    layer_out= Add()([conv3])
    layer_out = Add()([data])
    layer_out = Activation('relu')(layer_out)
    return layer_out

In [106]:
from keras.models import Model
from keras.layers import *
from keras.regularizers import *
from keras.optimizers import RMSprop

#define model input
visible = Input(shape=(224,224, 3))
layer1 = resnet(visible,64,3)
layer2 = resnet(layer1,128,1)
layer4 = resnet(layer2,256,1)
layer5 = resnet(layer4,256,2)
layer6 = resnet(layer5,512,2)
layer7 = resnet(layer6,512,2)
layer8 = resnet(layer7,1024,2)
layert = Dropout(0.5)(layer8)
layer9 = resnet(layert,2048,2)
layert2 = Dropout(0.5)(layer9)
layer10 = resnet(layert2,4096,2)
x = GlobalAveragePooling2D()(layer10)
x = Dropout(0.7)(x)
#flatten = Flatten()(layer9)
den = Dense(2048,activation = 'sigmoid')(x)
final = Dense(5, activation="softmax")(den)
#create model
model = Model(inputs=visible, outputs=final)

#compiling model
lr = 1e-5
decay = 1e-7 #0.0
optimizer = RMSprop(lr=lr, decay=decay)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
# summarize model
model.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_47 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_247 (Conv2D)         (None, 75, 75, 64)        256       
                                                                 
 batch_normalization_140 (Ba  (None, 75, 75, 64)       256       
 tchNormalization)                                               
                                                                 
 add_34 (Add)                (None, 75, 75, 64)        0         
                                                                 
 activation_114 (Activation)  (None, 75, 75, 64)       0         
                                                                 
 conv2d_251 (Conv2D)         (None, 75, 75, 128)       8320      
                                                          

In [120]:

history = model.fit(X_train,y_train,steps_per_epoch = 10,shuffle = True,epochs = 3)


Epoch 1/3
10/10 [==============================] - 5s 544ms/step - loss: 0.9748 - accuracy: 0.6452
Epoch 2/3
10/10 [==============================] - 5s 515ms/step - loss: 0.9208 - accuracy: 0.6989
Epoch 3/3
10/10 [==============================] - 5s 534ms/step - loss: 0.9577 - accuracy: 0.6129


In [138]:
from keras.utils import to_categorical
users = {'ben_afflek':0,'elton_john':1,'jerry_seinfeld':2,'madonna':3,'mindy_kaling':4}
images = []
label = []
for dirname, _, filenames in os.walk('/Users/91818/Downloads/ML_Project/archive/data/test/'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cropFaces(img)
        if(img.shape[0] == 0 or img.shape[1] == 0 or len(img) == 0):
            continue
        img = cv2.resize(img,(224,224))/255
        images.append(img)
        lbl = dirname.split('/')[-1]
        label.append(users[lbl])

df = pd.DataFrame()   #we are creating this dataframe to shuffle the input dataset.
df['image'] = images
df['label'] = label
df = df.sample(frac = 1)
X_test=  np.array(df.iloc[:,0].tolist())
y_test = np.array(df.iloc[:,1].tolist())
y_test = to_categorical(y_test)


1/1 [==============================] - 0s 18ms/step


In [139]:
len(X_test)
len(y_test)

5

In [140]:
score = model.evaluate(X_test, y_test ,verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

1/1 [==============================] - 0s 74ms/step - loss: 0.9446 - accuracy: 0.4000
Test Loss: 0.9445632696151733
Test accuracy: 0.4000000059604645


In [188]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm


In [189]:
model.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_47 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_247 (Conv2D)         (None, 75, 75, 64)        256       
                                                                 
 batch_normalization_140 (Ba  (None, 75, 75, 64)       256       
 tchNormalization)                                               
                                                                 
 add_34 (Add)                (None, 75, 75, 64)        0         
                                                                 
 activation_114 (Activation)  (None, 75, 75, 64)       0         
                                                                 
 conv2d_251 (Conv2D)         (None, 75, 75, 128)       8320      
                                                          